# How to Use This Notebook
This notebook shows how to set up a jupyter notebook to load resources that were designed for use in the hutch-python ipython shell.
You should follow the instructions in the next cell to set up the server process, and then cherry-pick the following cells as needed based on what you want the user to be able to do in the notebook. Feel free to customize and experiment as needed.

This was written assuming usage in xpp. You should be able to do exactly the same thing for other instruments, replacing references to xpp items with the relevent ones from another hutch.

# Setting up the jupyter server
First, you need to source xppenv (or xcsenv or etc.) to get the right env variables for the jupyter kernel.
This makes the jupyter kernel find the xpp beamline files and makes the DAQ work.
Note that this locks your terminal session to being compatible with exactly one particular version of the DAQ.
```
zlentz@psbuild-rhel7-02:~$ source /cds/group/pcds/pyps/apps/hutch-python/xpp/xppenv
Loading NFS python env pcds-5.8.0
```

This also lets you register the python env for use in the notebook if you haven't already done this:
```
(pcds-5.8.0)zlentz@psbuild-rhel7-02:~$ python -m ipykernel install --user --name=pcds-5.8.0
Installed kernelspec pcds-5.8.0 in /cds/home/z/zlentz/.local/share/jupyter/kernels/pcds-5.8.0
```

Then you can run the notebook from the same session:
```
(pcds-5.8.0)zlentz@psbuild-rhel7-02:~$ jupyter-notebook
```

If you want to control the DAQ, then the jupyter-nootback process must be run on the same linux host that the DAQ is running on.

## Making sure it works
This cell should import without errors and there should be lots of "xpp" on the paths. Mine is slightly different since I'm running on psbuild instead of xpp-daq:

In [1]:
import os
import pydaq
import xpp
print(os.environ["HAPPI_CFG"])
print(os.environ["PYTHONPATH"])
print(os.environ["LD_LIBRARY_PATH"])
print(xpp.__file__)
print(pydaq.__file__)

/cds/group/pcds/pyps/apps/hutch-python/device_config/happi.cfg
/cds/group/pcds/pyps/apps/hutch-python/xpp:/cds/group/pcds/pyps/apps/hutch-python/xpp/dev/devpath:/cds/group/pcds/pyps/apps/hutch-python/xpp/../common/dev/devpath:/reg/g/pcds/dist/pds/xpp/current/build/pdsapp/lib/x86_64-rhel7-opt:/reg/g/pcds/dist/pds/xpp/ami-current/build/ami/lib/x86_64-rhel7-opt
:/reg/g/pcds/dist/pds/xpp/ami-current/build/ami/lib/x86_64-rhel7-opt:/reg/g/pcds/dist/pds/xpp/ami-current/build/gsl/lib/x86_64-rhel7-opt:/reg/g/pcds/dist/pds/xpp/ami-current/build/qt/lib/x86_64-rhel7-opt:/reg/g/pcds/dist/pds/xpp/current/build/pdsapp/lib/x86_64-rhel7-opt:/reg/g/pcds/dist/pds/xpp/current/build/psalg/lib/x86_64-rhel7-opt:/reg/g/pcds/dist/pds/xpp/current/build/pds/lib/x86_64-rhel7-opt:/reg/g/pcds/dist/pds/xpp/current/build/pdsdata/lib/x86_64-rhel7-opt:/reg/g/pcds/dist/pds/xpp/current/build/offlinedb/lib/x86_64-rhel7-opt
/cds/group/pcds/pyps/apps/hutch-python/xpp/xpp/__init__.py
/reg/g/pcds/dist/pds/xpp/current/build/pd

# Session Setup
It's probably a good idea to do the setup in the same order as done in the full load, even if you skip steps. Below is the order and the current best way to do each step independently.

## Virtual Module Cache (xpp.db)
This is a utility that lets e.g. the beamline file import objects that are created automatically by importing them from xpp.db.

In [2]:
from hutch_python.cache import LoadCache
cache = LoadCache("xpp.db")

## Ophyd (EPICS) settings
These settings are used to deal with having tens of thousands of PV connections in one session. You might not need this for small sessions.

In [3]:
from hutch_python.ophyd_settings import setup_ophyd
setup_ophyd()

## Run Engine
This is required if you want to do bluesky scans. The BEC sets up the text output table.
TODO: figure out how to get the jupyter plot in (see example at https://github.com/bluesky/tutorials/blob/main/bluesky-tutorial-utils/bluesky_tutorial_utils/beamline_configuration.py)

In [4]:
from bluesky import RunEngine
from bluesky.callbacks.best_effort import BestEffortCallback
RE = RunEngine({})
bec = BestEffortCallback()
bec.disable_plots()
RE.subscribe(bec)
cache(RE=RE, bec=bec)

## Calc utils
These are just the calculation utlities e.g. the be lens calcs

In [5]:
from pcdscalc.be_lens_calcs import *
from pcdscalc.common import *
from pcdscalc.diffraction import *

## DAQ
This is the classic LCLS1 daq. It ingests the RE for proper run closing during a scan.
TODO: add instructions for LCLS2 daq setup

In [6]:
from pcdsdaq.daq.original import Daq
daq = Daq(RE=RE, hutch_name="xpp")
cache(daq=daq)

## Bluesky Plans
This makes the standard bluesky plans (scans) available in curated namespaces.

- bp: classic bluesky plans
- bps: plan stubs, building blocks for bluesky plans
- bpp: plan preprpocessors, functions that modify other plans
- re: functions that run as-is without needing to pass them into RE

This also adds some runnable functions onto the daq object, e.g. the daq plans like daq.ascan, for example.

In [7]:
from hutch_python.plan_wrappers import initialize_wrapper_namespaces
from hutch_python import plan_defaults

re = initialize_wrapper_namespaces(
    RE=RE,
    plan_namespace=plan_defaults.plans,
    daq=daq,
)
bp = plan_defaults.plans
bps = plan_defaults.plan_stubs
bpp = plan_defaults.preprocessors
cache(re=re, bp=bp, bps=bps, bpp=bpp)

## Scan PVs
These are the PVs that are used to make run tables in the logbook that update based on scan progress.

In [8]:
from pcdsdaq.scan_vars import ScanVars
scan_pvs = ScanVars(
    "XPP:SCAN",
    name="scan_pvs",
    RE=RE,
)
scan_pvs.enable()
cache(scan_pvs=scan_pvs)

## Happi Devices
If you want to pick a few beamline devices to include, here's how

In [9]:
from happi import Client
client = Client.from_config()
xpp_sb3_pim = client.load_device(name="xpp_sb3_pim")

Can not enforce type to match attribute prefix_det as it does not exist on the container.


## Questionnaire Objects
Create objects that are defined in the questionnaire

In [10]:
from types import SimpleNamespace
from hutch_python.qs_load import get_qs_objs
qs_objs = get_qs_objs("xppc00121")
qs = SimpleNamespace(**qs_objs)
print(qs)

Failed to create a happi database entry from the questionnaire device: motors:8. KeyError: 'pvbase'
Failed to create a happi database entry from the questionnaire device: motors:5. KeyError: 'pvbase'
Failed to create a happi database entry from the questionnaire device: motors:6. KeyError: 'pvbase'
Failed to create a happi database entry from the questionnaire device: motors:3. KeyError: 'pvbase'
Failed to create a happi database entry from the questionnaire device: motors:7. KeyError: 'pvbase'
Failed to create a happi database entry from the questionnaire device: motors:2. KeyError: 'pvbase'
Failed to create a happi database entry from the questionnaire device: motors:12. KeyError: 'pvbase'
Failed to create a happi database entry from the questionnaire device: motors:11. KeyError: 'pvbase'
Failed to create a happi database entry from the questionnaire device: motors:10. KeyError: 'pvbase'
Failed to create a happi database entry from the questionnaire device: motors:4. KeyError: 'pvbas

namespace(jungfrau_y=IMS(HXR:PRT:01:MMS:02, name=jungfrau_y), g2yaw=IMS(XPP:USR:PRT:MMS:21, name=g2yaw), g1_pi=Newport(XPP:USR:MMN:01, name=g1_pi), sam_chi=IMS(HXR:PRT:01:MMS:01, name=sam_chi), k_eta=IMS(XPP:GON:MMS:09, name=k_eta), jungfrau_x=Newport(XPP:USR:PRT:MMN:05, name=jungfrau_x), t5chi=IMS(HXR:PRT:01:MMS:07, name=t5chi), prism_th=Newport(XPP:USR:MMN:07, name=prism_th), t2th=IMS(XPP:USR:MMS:30, name=t2th), crl_y=IMS(XPP:USR:PRT:MMS:22, name=crl_y), bs_y=Newport(XPP:USR:PRT:MMN:02, name=bs_y), bs_x=Newport(XPP:USR:PRT:MMN:01, name=bs_x), g2x=IMS(XPP:USR:PRT:MMS:17, name=g2x), sam_th=IMS(XPP:USR:PRT:MMS:32, name=sam_th), t1chi=IMS(XPP:USR:MMS:29, name=t1chi), g1_x=IMS(XPP:USR:MMS:22, name=g1_x), g2y=IMS(XPP:USR:PRT:MMS:18, name=g2y), t3x=IMS(XPP:USR:MMS:31, name=t3x), t4th=IMS(HXR:PRT:01:MMS:06, name=t4th), t4x=IMS(HXR:PRT:01:MMS:05, name=t4x), zyla_z=IMS(XPP:USR:PRT:MMS:25, name=zyla_z), d3x=IMS(HXR:PRT:01:MMS:10, name=d3x), prism_y=Newport(XPP:USR:MMN:04, name=prism_y), k_phi=I

## Beamline load
The classic

In [11]:
from xpp.beamline import *

Failed to load pyami detectors after 2.11 s
Successfully loaded Event Sequencers in 0.01 s
Failed to load K mono after 0.01 s


0.0


Successfully loaded fake_delay in 0.00 s
Failed to load xpp_lens after 0.00 s
Successfully loaded GON motors XYZ in 0.05 s
Successfully loaded GON motors Phi-Z in 0.03 s
Successfully loaded xpp_gon_kappa in 0.40 s
Failed to load Combine gon objects after 0.00 s
Failed to load gon and kappa after 0.00 s
Successfully loaded Roving Spectrometer in 0.00 s
Successfully loaded Noplot ascan in 0.00 s
Successfully loaded Mode change in 0.00 s
Failed to load FS11 & FS14 lxt & lxt_ttc after 0.01 s
Successfully loaded Delay Scan in 0.00 s
Successfully loaded Laser Shutters - cp, lp & ep in 0.01 s
Failed to load Create Aliases after 0.00 s
Failed to load add pp motors after 0.02 s
Successfully loaded add crl motors in 0.05 s
Successfully loaded LIB SmarAct in 0.11 s
Successfully loaded AMI auto save in 0.00 s
Failed to load add laser motor groups after 0.00 s
Successfully loaded enable scan table scientific notation in 0.00 s
Successfully loaded Time tool target in 0.03 s
Successfully loaded autor

## Experiment file load
The other classic

In [12]:
from hutch_python.exp_load import get_exp_objs
x = get_exp_objs("c00121", ask_on_failure=False)

Successfully loaded c00121 in 0.00 s
